In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras as ks
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# Load pre-shuffled MNIST data into train and test sets
(x_train_raw, y_train_raw), (x_test_raw, y_test_raw) = ks.datasets.mnist.load_data()

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(x_train_raw[2], cmap="gray_r")

In [ ]:
x_train = x_train_raw/255.0
x_test = x_test_raw/255.0

In [ ]:
enc = OneHotEncoder()

y_train = enc.fit_transform(y_train_raw.reshape(-1, 1)).toarray()
y_test = enc.transform(y_test_raw.reshape(-1, 1)).toarray()

In [ ]:
class AutoEncoder(ks.Model):
    
    def __init__(self, name="autoencoder", **kwargs):
        super(AutoEncoder, self).__init__(name=name, **kwargs)
        self.encoder = ks.Sequential([
            ks.layers.Flatten(),
            ks.layers.Dense(512, activation="relu"),
            ks.layers.Dense(256, activation="sigmoid")
        ])
        self.decoder = ks.Sequential([
            ks.layers.Dense(784, activation="sigmoid"),
            ks.layers.Reshape((28,28))
        ])
     
    def call(self, inputs):
        return self.decoder(self.encoder(inputs))

In [ ]:
np.random.seed(666)
tf.random.set_seed(666)

auto = AutoEncoder()

auto.compile(loss='mean_squared_error', optimizer='adam')
auto.fit(x=x_train, y=x_train, batch_size=64, epochs=20, validation_data=(x_test, x_test))

In [ ]:
x_train_enc = auto.encoder(x_train)
x_train_dec = auto.decoder(x_train_enc)
x_test_enc = auto.encoder(x_test)
x_test_dec = auto.decoder(x_test_enc)

plt.imshow(x_train_dec[2], cmap="gray_r")

In [ ]:
np.random.seed(666)
tf.random.set_seed(666)

model = ks.models.Sequential()
model.add(ks.layers.Dense(128, activation=ks.activations.relu))
model.add(ks.layers.Dense(64, activation=ks.activations.sigmoid))
model.add(ks.layers.Dense(10, activation=ks.activations.softmax))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x=x_train_enc, y=y_train, batch_size=32, epochs=20)

In [ ]:
test_loss, test_acc = model.evaluate(x=x_test_enc, y=y_test)

print('Test')
print('Loss: %6.6f' % test_loss)
print('Accuracy: %0.4f' % test_acc)